#### Import modules

In [1]:
import numpy as np  # Matrix  manipulation
import os  # Filesystem support
import pandas as pd  # Dataframe
import Levenshtein  # Calculating edit distance

#### Set project folders

In [2]:
PROJECT_FOLDER = os.path.dirname(os.path.dirname(os.getcwd()))
RAW_DATA_FOLDER = os.path.join(PROJECT_FOLDER, 'data', 'raw')
PROCESSED_DATA_FOLDER = os.path.join(PROJECT_FOLDER, 'data', 'processed')
FINAL_DATA_FOLDER = os.path.join(PROJECT_FOLDER, 'data', 'final')

#### Pandas settings

In [3]:
pd.set_option("display.precision", 3)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.max_rows", 50)

#### Load dataframe

In [4]:
DATA =  os.path.join(
    PROCESSED_DATA_FOLDER, 
    'experiment_1',
    'data_processed.feather'
)
df = pd.read_feather(DATA)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3094 entries, 0 to 3093
Columns: 443 entries, Start Date to Id
dtypes: float64(339), int64(8), object(96)
memory usage: 10.5+ MB


#### Drop timing variables and rename column headers

In [5]:
to_drop = [
    'First Click',
    'Last Click',
    'Click Count',
]

columns_to_drop = [x for x in df.columns for y in to_drop if y in x]
df = df.drop(columns_to_drop, axis=1).rename(lambda x: x.capitalize(), axis=1)

####  Rename columns

In [6]:
df = df.rename(
    columns={
        df.columns[2]: "Identification_timer",
        df.columns[3]: "Instructions_timer",
        df.columns[4]: "Trial_text",
        df.columns[5]: "No_intervention_timer",
        df.columns[6]: "Intervention_timer",
        df.columns[7]: "Participation_answer",
        df.columns[168]: "Enjoyment",
        df.columns[169]: "Motivation",
        df.columns[170]: "Future_motivation",
        df.columns[171]: "Latin",
        df.columns[172]: "Mobile_device",
        df.columns[173]: "Touch_typing",
        df.columns[174]: "Hours_keyboard",
        df.columns[175]: "Age",
        df.columns[176]: "Female",
        df.columns[177]: "Education",
        df.columns[178]: "Comments",
    })

#### Relabel column headers

In [7]:
counter = 0
for column, title in enumerate(df.columns):
    if 'page submit' in title:
        counter += 1
        df.rename(columns={df.columns[column]:f'Q{counter}_timer'}, inplace=True)
    elif 'please enter' in title:
        df.rename(columns={df.columns[column]:f'Q{counter}_answer'}, inplace=True)

#### Make main effect columns

In [8]:
cols = ['Performance_pay', 'Leadership_technique', 'Clarification']
df[cols] = df['Treatment_str'].str.split('+', expand=True)

#### Reorder treatment coding  

In [9]:
treat_order = pd.Index([
    'No piece rate + No message',
    'No piece rate + Praise',
    'No piece rate + Reference point',
    'Low piece rate + No message',
    'Low piece rate + Praise',
    'Low piece rate + Reference point',
    'High piece rate + No message',
    'High piece rate + Praise',
    'High piece rate + Reference point',
    'Low piece rate + No message + Clarification',
    'Low piece rate + Praise + Clarification',
    'High piece rate + No message + Clarification',
    'High piece rate + Praise + Clarification',
    ], dtype='object')

df['Treatment'] = df['Treatment_str']\
    .astype('category')\
    .cat.set_categories(treat_order)\
    .cat.codes

treat_table = df\
    .groupby(['Treatment_str'])['Treatment']\
    .first()\
    .to_frame()\
    .reindex(treat_order)

treat_table

,Treatment
No piece rate + No message,0
No piece rate + Praise,1
No piece rate + Reference point,2
Low piece rate + No message,3
Low piece rate + Praise,4
Low piece rate + Reference point,5
High piece rate + No message,6
High piece rate + Praise,7
High piece rate + Reference point,8
Low piece rate + No message + Clarification,9


#### Make treatment string for each main effect 

In [10]:
df['No_message'] = df['Treatment'].isin([0, 3, 6, 9, 11]).astype(int)
df['Praise'] = df['Treatment'].isin([1, 4, 7, 10, 12]).astype(int)
df['Reference_point'] = df['Treatment'].isin([2, 5, 8]).astype(int)
df['No_piece_rate'] = df['Treatment'].isin([0, 1, 2]).astype(int)
df['Low_piece_rate'] = df['Treatment'].isin([3, 4, 5, 9, 10]).astype(int)
df['High_piece_rate'] = df['Treatment'].isin([6, 7, 8, 11, 12]).astype(int)
df['Clarification'] = df['Treatment'].isin([9, 10, 11, 12]).astype(int)

#### Convert timeseries columns

In [11]:
timseries = ['Start date', 'End date']
for series in timseries:
    df[series] = pd.to_datetime(df[series], infer_datetime_format=True)

df['Duration_survey'] = (df['End date'] - df['Start date']).dt.seconds
df['Duration_work'] = df['End_time'] - df['Start_time']

df = df.drop(df.columns[[0, 1, 185, 186]], axis=1)

#### Identify subjects for which the timer did not work (i.e. 10 min. +/- 5%)

In [12]:
low_limit = (df['Duration_work'] < 600 * 0.95)
up_limit = (df['Duration_work'] > 600 * 1.05)
broken_timer = (low_limit | up_limit)

broken_columns = ['Treatment', 'Duration_work', 'Counter']
df_broken = df.loc[broken_timer, broken_columns]\
    .sort_values(by='Duration_work')\
    .rename(lambda x: x.capitalize().replace('_', ' '), axis=1)

df_broken

,Treatment,Duration work,Counter
713,1,394.0,15
2596,2,503.0,59
2027,12,634.0,35
2106,12,636.0,0
660,0,640.0,20
695,7,642.0,14
392,3,654.0,13
3047,2,655.0,13
577,6,670.0,3
2783,2,675.0,12


#### Remove subjects for which the timer did not work

In [13]:
df = df[~broken_timer].reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3076 entries, 0 to 3075
Columns: 198 entries, Identification_timer to Duration_work
dtypes: float64(87), int64(14), int8(1), object(96)
memory usage: 4.6+ MB


#### Type convert string variables to integer variables

In [14]:
genders = {'Male': 0, 'Female': 1}
df['Female'] = df['Female'].replace(genders)

device = {'No': 0, 'Yes': 1}
df['Mobile_device'] = df['Mobile_device'].replace(device)

#### Type convert categorial variables to integer variables

In [15]:
categorial = [
    'Enjoyment', 
    'Motivation', 
    'Future_motivation', 
    'Latin', 
    'Touch_typing',
]
for cats in categorial:
    df[cats] = df[cats].astype('category').cat.codes + 1

#### Generate binary knowledge of latin variable

In [16]:
df['Latin_med'] = (df['Latin'] > df['Latin'].median()).astype(int)

#### Recode education variable

In [17]:
education_ordered = pd.Index([
    u'High School / GED',
    u'Some College',
    u'2-year College Degree',
    u'4-year College Degree',
    u'Masters Degree',
    u'Doctoral Degree',
], dtype='object')

to_replace = {'Professional Degree (JD, MD)': 'Doctoral Degree'}
df['Education'] = df['Education']\
    .replace(to_replace)\
    .astype('category')\
    .cat.set_categories(education_ordered)\
    .cat.codes\
    .pipe(lambda x: x+1)

#### Remove leading/trailing whitespace and linebreaks

In [18]:
def remove_whitespace(x):
    if isinstance(x, str):
        return x.strip().replace('\n', ' ').replace('\r', '')
    else:
        return x

df = df.applymap(remove_whitespace)

#### Replace empty strings with nan values

In [19]:
answer_cols = [f'Q{x}_answer' for x in range(1, 81)]
df.loc[:, answer_cols] = df.loc[:, answer_cols].replace({'': np.nan})

#### Count non-empty answers (nb.: the last submission is not counted)

In [20]:
def count_non_empty(row: pd.Series) ->int:
    """
    Count actual number of submitted fragments.
    
    This function counts only non-empty submissions. 
    It takes account of the fact that workers
    skip fragments.
    """
    raw_submits = row['Counter']
    answer_columns = [f'Q{x}_answer' for x in range(1, raw_submits + 1)]
    return row[answer_columns].count()


df['Counter_real'] = df.apply(count_non_empty, axis=1)

#### Display dataframe section with submitted fragments and counter

In [21]:
mask = df['Counter'].between(20, 23)
df.loc[mask, answer_cols[20:24] + ['Counter', 'Counter_real']]

,Q21_answer,Q22_answer,Q23_answer,Q24_answer,Counter,Counter_real
0,ego aduc nulli nisi provecto iam credidid; mih...,nomen missu,None,None,21,21
15,ego adhur nulli nisi provecto iam credidi: mih...,nomen missum non esset. Probum principem fieri.,None,None,21,21
16,ego adhuc nulli nisi provecto iam credidi; mih...,nomen missum non esset. Probum prinicipem fieri.,regenas legiones accepit. credebat cum gratula...,"oppressit, paucis",23,23
22,ego adhuc nulli nisi provecto iam credidi; mih...,nomen missum non esset. Probum principem fieri.,regendas legiones accepit. credebat cum gratu...,None,22,22
25,ego adhuc nulli nisi provecto iam credidi; mih...,None,None,None,20,20
...,...,...,...,...,...,...
3053,ego adhuc nulli nisi provecto iam credidi; mih...,nomen missum non esset. Probum prinipem fieri.,regendas legiones accepit. credebat cum gratul...,oppr,23,23
3054,None,None,None,None,20,20
3056,ego adhue nulli nisi provecto iam credidi; mih...,nomen missum non esset. Probum principem fi,None,None,21,21
3058,ego adhue nulli nisi provecto iam credidi mihi...,None,None,None,21,21


#### Generate columns with counter for emty submissions and share of empty submissions

In [22]:
df['Counter_empty'] = df['Counter'] - df['Counter_real']
df['Counter_quota'] = df['Counter_empty'] / df['Counter']

#### Make dataframe with empty columns for soluton, edit distance and edit ratio, then join with main dataframe

In [23]:
edit_cols = [
    col for x in range(1, 81)
    for col in (f'Q{x}_solution', f'Q{x}_distance', f'Q{x}_editratio')
]

edit_frame = pd.DataFrame(index=df.index, columns=edit_cols)
df = df.merge(edit_frame, left_index=True, right_index=True)

#### Make columns with correct solutions

In [24]:
FRAGMENTS = os.path.join(PROCESSED_DATA_FOLDER, 'experiment_1', 'fragments.xlsx')

df_fragments = pd.read_excel(
    FRAGMENTS,
    header=None,
    names=["Keyword", "Solution"],
    engine='openpyxl',
    )\
    .applymap(lambda x: x.strip())\
    .drop('Keyword', axis=1)

for index, row in df_fragments.iterrows():
    df[f'Q{index + 1}_solution'] = row['Solution']

#### Calculate Levenshtein edit distance 

In [25]:
column_pairs = [(f'Q{x}_answer', f'Q{x}_solution') for x in range(1, 81)]
df['Wildcard_count'] = np.nan

for index, row in df.iterrows():

    number_answers_submitted = row.at['Counter']
    columns_to_check = column_pairs[:number_answers_submitted]
    wildcard_count = 0
    
    for number, (answer_col, solution_col) in enumerate(columns_to_check, 1):
        answer = row[answer_col]
        solution = row[solution_col]

        if isinstance(answer, str):
            edit_distance = Levenshtein.distance(answer, solution)
            edit_ratio = (edit_distance - answer.count('?')) / max(len(answer), len(solution))
            wildcard_count += answer.count('?')
            
        else:
            edit_distance = np.NAN
            edit_ratio = np.NAN

        dist_column = f'Q{number}_distance'
        ratio_column = f'Q{number}_editratio'

        df.at[index, dist_column] = edit_distance
        df.at[index, ratio_column] = edit_ratio
    
    df.at[index, 'Wildcard_count'] = wildcard_count

#### Manually inspecting answers and evaluations for submitted fragments

In [26]:
x = 15
cols = [
    f'Q{x}_answer', 
    f'Q{x}_solution', 
    f'Q{x}_distance',
    f'Q{x}_editratio', 
]

with pd.option_context("display.max_colwidth", 120):
    display(df.loc[(df['Counter'] >= x), cols].head(15))

,Q15_answer,Q15_solution,Q15_distance,Q15_editratio
0,?elicitas solita. Nam Aegyptum statim recepit atque.,"felicitas solita. Nam Aegyptum statim recepit atque,",2,0.0192
1,lelicitas solita. Nam Aegyptum statim recepit atque.,"felicitas solita. Nam Aegyptum statim recepit atque,",2,0.0385
2,"felicitas solita. Nam Aegyptum statim recepit atque,","felicitas solita. Nam Aegyptum statim recepit atque,",0,0
6,felicitas solita. Nam Aegyptum statim recepit atque.,"felicitas solita. Nam Aegyptum statim recepit atque,",1,0.0192
7,felicitas solita. Nam Aegyptum statim recepit atque.,"felicitas solita. Nam Aegyptum statim recepit atque,",1,0.0192
8,felicitas solita. Nam Aegyptum statim recepit atque,"felicitas solita. Nam Aegyptum statim recepit atque,",1,0.0192
9,"felicitas solita. Nam Aegyptum statim recepit atque,","felicitas solita. Nam Aegyptum statim recepit atque,",0,0
10,"felicitas solita. Nam Aegyptum statim recepit atque,","felicitas solita. Nam Aegyptum statim recepit atque,",0,0
11,felicitas solita. nam aegyptum statim recepit atque,"felicitas solita. Nam Aegyptum statim recepit atque,",3,0.0577
12,felicitas solita. Nam Aehyptum statim recepit atque.,"felicitas solita. Nam Aegyptum statim recepit atque,",2,0.0385


#### Order dataframe columns

In [27]:
cols = [
    'Q{0}_answer',
    'Q{0}_solution',
    'Q{0}_distance',
    'Q{0}_editratio',
    'Q{0}_timer',
]

ques_cols = [elem.format(x) for x in range(1, 81) for elem in cols]
main = list(df.columns.difference(ques_cols))
df = df.reindex(main + ques_cols, axis=1)

#### Save dataframe

In [28]:
path = os.path.join(FINAL_DATA_FOLDER, 'experiment_1')
df.to_csv(os.path.join(path, 'data_final.csv'))
df.to_feather(os.path.join(path, 'data_final.feather'))

In [29]:
!jupyter nbconvert --output-dir='./docs' --to html 1_make_dataset.ipynb

[NbConvertApp] Converting notebook 1_make_dataset.ipynb to html
[NbConvertApp] Writing 655356 bytes to docs/1_make_dataset.html
